# 🧮 Interactive Maths 
Game Matematika Sederhana dengan Hand Number Detection

In [8]:
# Import library yang dibutuhkan
import cv2
import mediapipe as mp
import numpy as np
import os
import random

In [9]:
# Fungsi untuk generate soal matematika
def generate_math_problem():
    operators = ['+', '-', '*', '/']
    operator = random.choice(operators)
    
    if operator == '+':
        # Penjumlahan (hasil 0-10)
        result = random.randint(0, 10)
        num2 = random.randint(0, result)
        num1 = result - num2
    elif operator == '-':
        # Pengurangan (hasil 0-10)
        num1 = random.randint(0, 10)
        num2 = random.randint(0, num1)
    elif operator == '*':
        # Perkalian (hasil 0-10)
        result = random.randint(0, 10)
        factors = [i for i in range(1, 11) if result % i == 0]
        num1 = random.choice(factors)
        num2 = result // num1
    else:
        # Pembagian (hasil 0-10, tanpa sisa)
        num2 = random.randint(1, 10)
        num1 = random.randint(0, 10) * num2
    
    problem = f"{num1} {operator} {num2}"
    answer = eval(problem)
    return problem, float(answer)

# State untuk game
class GameState:
    def __init__(self):
        self.current_problem = None
        self.current_answer = None
        self.problems_count = 0
        self.correct_answers = 0
        self.total_problems = 5
        self.score = 0  # Total score
        self.time_per_question = 7  # 5 seconds per question
        self.timer = 0  # Current timer value
        self.last_time = 0  # For tracking time
        self.current_answer_given = None  # Track user's last answer
        self.feedback_delay = 60  # Frames to show feedback (60 frames ≈ 2 seconds)
        self.show_feedback = False
        self.game_over_delay = 120  # 4 seconds delay for final score
        self.show_thank_you = False
        self.final_score_delay = 90  # 3 seconds for thank you message
        
    def new_problem(self):
        if self.problems_count < self.total_problems:
            self.current_problem, self.current_answer = generate_math_problem()
            self.timer = self.time_per_question
            self.last_time = cv2.getTickCount()
            return True
        return False
    
    def update_timer(self):
        if self.timer > 0:
            current_time = cv2.getTickCount()
            elapsed = (current_time - self.last_time) / cv2.getTickFrequency()
            self.timer = max(0, self.time_per_question - elapsed)
            return self.timer > 0
        return False
    
    def check_answer(self, user_answer):
        if self.timer > 0:  # Only check if there's still time
            if abs(user_answer - self.current_answer) < 0.01:
                self.score += 20  # Add 20 points for correct answer
                return True
        return False

In [10]:
# Daftar Indeks Landmark Tangan
JEMPOL = [1, 2, 3, 4]  # Jari Jempol
TELUNJUK = [5, 6, 7, 8]  # Jari Telunjuk
JARI_TENGAH = [9, 10, 11, 12]  # Jari Tengah
JARI_MANIS = [13, 14, 15, 16]  # Jari Manis
JARI_KELINGKING = [17, 18, 19, 20]  # Jari Kelingking

# Fungsi untuk mendeteksi angka dari posisi jari (diperbarui)
def detect_finger_number(hand_landmarks, handedness):
    fingers = []
    
    # Thumb detection - different logic for left and right hands
    thumb_tip = hand_landmarks.landmark[JEMPOL[-1]]
    thumb_base = hand_landmarks.landmark[JEMPOL[0]]
    thumb_ip = hand_landmarks.landmark[JEMPOL[2]]  # IP joint of thumb
    
    # Improve thumb detection accuracy
    if handedness == 'Right':
        fingers.append(thumb_tip.x < thumb_ip.x)
    else:
        fingers.append(thumb_tip.x > thumb_ip.x)
    
    # Other fingers - improved detection
    finger_lists = [TELUNJUK, JARI_TENGAH, JARI_MANIS, JARI_KELINGKING]
    for finger in finger_lists:
        tip = hand_landmarks.landmark[finger[-1]]
        pip = hand_landmarks.landmark[finger[-2]]  # Get PIP joint
        dip = hand_landmarks.landmark[finger[-3]]  # Get DIP joint
        
        # Finger is counted if tip is higher than both PIP and DIP
        fingers.append(tip.y < pip.y and tip.y < dip.y)
    
    return sum(fingers)

def combine_hand_numbers(left_count, right_count):
    """Combine numbers from both hands for numbers 0-10"""
    # If only one hand is detected
    if left_count == 0:
        return right_count
    if right_count == 0:
        return left_count
        
    # For numbers 6-10: left hand shows 5, right hand shows additional 1-5
    if left_count == 5:
        return 5 + right_count
    
    return min(10, left_count + right_count)

In [11]:
# Inisialisasi MediaPipe Hands dan Face Detection
mp_hands = mp.solutions.hands
mp_face = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7)
face_detection = mp_face.FaceDetection(min_detection_confidence=0.5)

# Fungsi untuk mengecek apakah tangan berinteraksi dengan tombol
def check_button_interaction(hand_landmarks, button_pos, button_size):
    if hand_landmarks:
        # Menggunakan titik telunjuk (index finger tip)
        index_finger = hand_landmarks.landmark[8]
        x, y = int(index_finger.x * width), int(index_finger.y * height)
        
        # Cek apakah telunjuk berada di area tombol
        button_x, button_y = button_pos
        button_w, button_h = button_size
        
        if (button_x < x < button_x + button_w and 
            button_y < y < button_y + button_h):
            return True
    return False

# Fungsi untuk menempatkan gambar ke dalam frame
def overlay_image(background, overlay, position):
    h, w = overlay.shape[:2]
    y, x = position
    
    if overlay.shape[2] == 4:  # Jika ada alpha channel
        # Memisahkan alpha channel
        overlay_rgb = overlay[:,:,:3]
        alpha = overlay[:,:,3] / 255.0
        
        # Membuat alpha channel 3D
        alpha_3d = np.stack([alpha, alpha, alpha], axis=2)
        
        # Menghitung area yang akan ditimpa
        roi = background[y:y+h, x:x+w]
        # Menggabungkan gambar dengan alpha blending
        result = (overlay_rgb * alpha_3d + roi * (1 - alpha_3d))
        background[y:y+h, x:x+w] = result
    else:  # Jika tidak ada alpha channel
        background[y:y+h, x:x+w] = overlay
    return background

In [12]:
# Membaca gambar asset
logo_path = os.path.join(os.getcwd(), 'asset', 'logo.png')
button_path = os.path.join(os.getcwd(), 'asset', 'buttonstart.png')
bgcard_path = os.path.join(os.getcwd(), 'asset', 'backgroundcard.png')

# Membaca gambar
logo = cv2.imread(logo_path, cv2.IMREAD_UNCHANGED)
button = cv2.imread(button_path, cv2.IMREAD_UNCHANGED)
bgcard = cv2.imread(bgcard_path, cv2.IMREAD_UNCHANGED)

# Cek apakah gambar berhasil dibaca
if logo is None or button is None or bgcard is None:
    print("Error: Tidak dapat membaca file gambar!")
    print(f"Logo path: {logo_path}")
    print(f"Button path: {button_path}")
    print(f"Background Card path: {bgcard_path}")
else:
    # Mengubah ukuran gambar
    logo = cv2.resize(logo, (200, 200))
    button = cv2.resize(button, (200, 60))
    bgcard = cv2.resize(bgcard, (300, 150)) 

    # Inisialisasi webcam
    cap = cv2.VideoCapture(0)
    
    # State untuk mengontrol tampilan
    is_game_started = False
    game_state = GameState()
    answer_display_time = 0
    answer_feedback = ""

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Gagal membaca frame dari webcam")
            break

        frame = cv2.flip(frame, 1)
        height, width = frame.shape[:2]

        # Konfigurasi text
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1.0
        thickness = 2
        text = game_state.current_problem

        # Konversi ke RGB untuk MediaPipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Deteksi tangan dan wajah
        hand_results = hands.process(rgb_frame)
        face_results = face_detection.process(rgb_frame)

        if not is_game_started:
            # Tampilan menu utama
            logo_x = (width - 200) // 2
            logo_y = 50
            button_x = (width - 200) // 2
            button_y = logo_y + 200 + 20

            frame = overlay_image(frame, logo, (logo_y, logo_x))
            frame = overlay_image(frame, button, (button_y, button_x))

            # Cek interaksi tangan dengan tombol
            if hand_results.multi_hand_landmarks:
                for hand_landmarks in hand_results.multi_hand_landmarks:
                    # Gambar landmark tangan
                    mp_drawing.draw_landmarks(
                        frame,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS)
                    
                    # Cek interaksi dengan tombol start
                    if check_button_interaction(
                        hand_landmarks,
                        (button_x, button_y),
                        (200, 60)
                    ):
                        is_game_started = True
        else:
            # Mendeteksi posisi wajah untuk penempatan background card
            if face_results.detections:
                for detection in face_results.detections:
                    # Mendapatkan bounding box wajah
                    bbox = detection.location_data.relative_bounding_box
                    
                    # Konversi ke koordinat pixel
                    face_x = int(bbox.xmin * width)
                    face_y = int(bbox.ymin * height)
                    face_w = int(bbox.width * width)
                    face_h = int(bbox.height * height)
                    
                    # Menyesuaikan posisi background card di atas kepala
                    bgcard_x = face_x - (bgcard.shape[1] - face_w) // 2
                    bgcard_y = face_y - bgcard.shape[0] - 20  # 20 pixel di atas kepala
                    
                    # Memastikan background card tetap dalam frame
                    bgcard_x = max(0, min(bgcard_x, width - bgcard.shape[1]))
                    bgcard_y = max(0, min(bgcard_y, height - bgcard.shape[0]))
                    
                    # Menempelkan background card
                    frame = overlay_image(frame, bgcard, (bgcard_y, bgcard_x))

                    # Generate soal baru jika belum ada
                    if game_state.current_problem is None:
                        game_state.new_problem()
                        answer_display_time = 0
                        answer_feedback = ""

                    if game_state.current_problem:
                        # Update timer
                        if not game_state.update_timer():
                            # Time's up - check answer and show feedback
                            if not game_state.show_feedback:
                                # Get user's final answer
                                final_answer = total_fingers if 'total_fingers' in locals() else 0
                                correct = final_answer == int(game_state.current_answer)
                                
                                if correct:
                                    game_state.score += 20
                                    answer_feedback = "Benar! +20"
                                else:
                                    answer_feedback = "Salah! +0"
                                
                                game_state.show_feedback = True
                                answer_display_time = game_state.feedback_delay

                            elif answer_display_time > 0:
                                feedback_color = (0, 255, 0) if "Benar" in answer_feedback else (0, 0, 255)
                                feedback_pos = (width//2 - 60, height//2)
                                
                                # Add black outline for better visibility
                                cv2.putText(frame, answer_feedback,
                                          feedback_pos, font, 1,
                                          (0, 0, 0), 3)
                                cv2.putText(frame, answer_feedback,
                                          feedback_pos, font, 1,
                                          feedback_color, 2)
                                
                                answer_display_time -= 1
                            else:
                                # Move to next question after delay
                                game_state.problems_count += 1
                                game_state.current_problem = None
                                game_state.show_feedback = False

                        # Display timer
                        timer_text = f"Waktu: {int(game_state.timer)}s"
                        cv2.putText(frame, timer_text,
                                  (width - 150, 30), font, 0.7,
                                  (0, 0, 255) if game_state.timer < 2 else (255, 255, 255), 2)
                        
                        # Display current score
                        score_text = f"Score: {game_state.score}"
                        cv2.putText(frame, score_text,
                                  (width - 150, 60), font, 0.7,
                                  (255, 255, 255), 2)

                        # Hitung ukuran text
                        (text_width, text_height), baseline = cv2.getTextSize(
                            text, font, font_scale, thickness)

                        # Hitung posisi tengah dalam background card
                        text_x = bgcard_x + (bgcard.shape[1] - text_width) // 2
                        text_y = bgcard_y + (bgcard.shape[0] + text_height) // 2

                        # Tambahkan text dengan outline putih untuk keterbacaan
                        cv2.putText(frame, text, (text_x, text_y), font,
                                  font_scale, (255, 255, 255), thickness + 1)
                        cv2.putText(frame, text, (text_x, text_y), font,
                                  font_scale, (0, 0, 0), thickness)

                        # Deteksi angka dari kedua tangan
                        left_count = 0
                        right_count = 0
                        if hand_results.multi_hand_landmarks:
                            for idx, hand_landmarks in enumerate(hand_results.multi_hand_landmarks):
                                handedness = hand_results.multi_handedness[idx].classification[0].label
                                count = detect_finger_number(hand_landmarks, handedness)
                                
                                # Draw hand landmarks with styles
                                mp_drawing.draw_landmarks(
                                    frame,
                                    hand_landmarks,
                                    mp_hands.HAND_CONNECTIONS,
                                    mp_drawing_styles.get_default_hand_landmarks_style(),
                                    mp_drawing_styles.get_default_hand_connections_style()
                                )
                                
                                # Track counts for each hand
                                if handedness == 'Left':
                                    left_count = count
                                else:
                                    right_count = count
                                
                                # Show individual hand counts (debug info)
                                hand_pos_y = 50 + (idx * 30)
                                cv2.putText(frame, f"{handedness}: {count}", 
                                          (10, hand_pos_y), font, 0.7, 
                                          (255, 255, 255), 2)
                            
                            # Combine numbers from both hands
                            total_fingers = combine_hand_numbers(left_count, right_count)
                            
                            # Show current answer (neutral color)
                            cv2.putText(frame, f"Jawaban: {total_fingers}", 
                                      (10, 110), font, 1, (255, 255, 255), 2)
                            
                            # Store the current answer
                            game_state.current_answer_given = total_fingers

                        # If no hands detected, use last stored answer
                        elif hasattr(game_state, 'current_answer_given'):
                            total_fingers = game_state.current_answer_given
                            cv2.putText(frame, f"Jawaban: {total_fingers}", 
                                      (10, 110), font, 1, (255, 255, 255), 2)

                        # Tampilkan feedback dengan warna sesuai (hanya saat waktu habis)
                        if answer_display_time > 0:
                            feedback_color = (0, 255, 0) if "Benar" in answer_feedback else (0, 0, 255)
                            feedback_pos = (width//2 - 60, height//2)
                            
                            # Add black outline for better visibility
                            cv2.putText(frame, answer_feedback,
                                      feedback_pos, font, 1,
                                      (0, 0, 0), 3)  # Outline
                            cv2.putText(frame, answer_feedback,
                                      feedback_pos, font, 1,
                                      feedback_color, 2)  # Text
                            
                            answer_display_time -= 1

                        # Game Over handling
                        if game_state.problems_count >= game_state.total_problems:
                            if not game_state.show_thank_you:
                                # Show final score first
                                final_score_text = f"Selesai! Score: {game_state.score}/100"
                                # Add black outline for better visibility
                                cv2.putText(frame, final_score_text,
                                          (width//4 - 10, height//2),
                                          font, 1.5, (0, 0, 0), 4)  # Outline
                                cv2.putText(frame, final_score_text,
                                          (width//4 - 10, height//2),
                                          font, 1.5, (0, 255, 0), 2)  # Text
                                
                                # Tampilkan tombol "Try Again"
                                try_again_x = (width - 200) // 2
                                try_again_y = height // 2 + 100
                                frame = overlay_image(frame, button, (try_again_y, try_again_x))

                                # Cek interaksi dengan tombol "Try Again"
                                if hand_results.multi_hand_landmarks:
                                    for hand_landmarks in hand_results.multi_hand_landmarks:
                                        if check_button_interaction(
                                            hand_landmarks,
                                            (try_again_x, try_again_y),
                                            (200, 60)
                                        ):
                                            # Reset game state
                                            game_state = GameState()
                                            is_game_started = False
                                            break
                            else:
                                # Clear background by creating blank frame
                                frame = np.zeros((height, width, 3), dtype=np.uint8)
                                
                                # Show thank you message
                                thank_you_text = "Terima Kasih!"
                                text_size = cv2.getTextSize(thank_you_text, font, 2, 3)[0]
                                text_x = (width - text_size[0]) // 2
                                text_y = height // 2
                                
                                # Add black outline for better visibility
                                cv2.putText(frame, thank_you_text,
                                          (text_x, text_y),
                                          font, 2, (0, 0, 0), 5)  # Outline
                                cv2.putText(frame, thank_you_text,
                                          (text_x, text_y),
                                          font, 2, (255, 255, 255), 3)  # Text
                                
                                if game_state.game_over_delay > 0:
                                    game_state.game_over_delay -= 1
                                else:
                                    break  # Exit program

        cv2.imshow('Interactive Maths', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    hands.close()
    face_detection.close()
    cap.release()
    cv2.destroyAllWindows()